In [2]:
# Set up your Python workspace
# Note: The STK Python API used in this lesson is
# only available with STK 12.1.
# If not installed then use pip to install it.
# pip install agi.stk<..ver..>-py3-none-any.whl
# If using an older version of STK then use win32api or Comtypes

from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.stkobjects import *

# When connected to STK via Python, while creating your variable, 
# using the Tab key after periods enables IntelliSense which displays 
# all of the options available off of the current interface. 
# In the next section you will start STK 

In [3]:
# NOTE FOR STK WEB: you can take advantage of STK/SDF SSO by changing 
# your script to connect to an active instance instead of creating a 
# new instance of STK:
# Connect to an an instance of STK12
# stk = STKDesktop.AttachToApplication()

# Create a new instance of STK12.
# Optional arguments set the application visible state and the user-control 
# (whether the application remains open after exiting python).
stk = STKDesktop.StartApplication(visible=True, userControl=True)
#Check your Task Manager to confirm that STK was called 

# Grab a handle on the STK application root.
root = stk.Root

# Recall that the AGStkObjectRoot object is at the apex of the STK Object Model. 
# The associated IAgStkObjecetRoot interface will provide the methods and properties to load 
# or create new scenarios and aaccess the Object Model Unit preferences. Through app you have 
# a pointer to the IAgUiApplication interface. How will you obtain a pointer to the IAgStkObjectRoot
# interface? According to IAgUiApplication documentation, the stk.GetObjectRoot() property returns 
# a new instance of the root object of the STK Object Model. 

In [4]:
# Check that the root object has been built correctly, check the type()

type(root)

# output will be 
# agi.stk12.stkobjects.AgStkObjectRoot

agi.stk12.stkobjects.AgStkObjectRoot

In [5]:
# Now that you have launched STK via the Python interface, 
# let's see if we can create a new scenario and set the time 
# period via Python. Create a new scenario, analysis period and 
# reset the animation time.

# 1. Create a new scenario.
# The next task is to create a scenario via the NewScenario method 
# of the IAgStkObjectRoot interface. According to the documentation, 
# the NewScenario method expects to be passed a string representing 
# the name of the scenario, but does not return anything.
root.NewScenario("Python_STK_Training")

scenario = root.CurrentScenario

# 2. Set the analytical time period.

scenario.SetTimePeriod('Today','+24hr')

# 3. Reset the animation time to the newly established start time.

root.Rewind();

In [6]:
# With a new scenario created, it's time to populate the scenario 
# with objects. Use the STK Python API

# 1. Add a target object to the scenario. 
target  = AgTarget(scenario.Children.New(AgESTKObjectType.eTarget,"GroundTarget"))

# 2. Move the Target object to a desired location.

target.Position.AssignGeodetic(50,-100,0)

In [7]:
# Take a moment to create a LEO satellite using Python and
# STK Connect commands, via the ExecuteCommand method.

# 3. Add a Satellite object to the scenario.

satellite  = AgSatellite(root.CurrentScenario.Children.New(AgESTKObjectType.eSatellite,"LeoSat"))

In [8]:
# Examine the below connect command before running. In it we will be using the
# Set State Classical connect command. Rather than manually setting the times
# we will use the define scenario times. Print them to confirm. 

print(scenario.StartTime)
print(scenario.StopTime)

# 4. Propagate the Satellite object's orbit.

root.ExecuteCommand('SetState */Satellite/LeoSat Classical TwoBody "' + str(scenario.StartTime) + '" "' + str(scenario.StopTime) + '" 60 ICRF  "' + str(scenario.StartTime) + '" 7200000.0 0.0 90 0.0 0.0 0.0');

19 Jan 2021 05:00:00.000
20 Jan 2021 05:00:00.000


In [9]:
# Now that the scenario is fully built, the final task is to extract data and perform a basic analysis.
# You have a scenario with a Target object and a Satellite object. 
# Determine when the Satellite object can access the Target object.
# Locate and manually enter code into Python to compute an access between 
# two STK Objects using the IAgStkObject interface. 
# HINT: Review the Python Code snippits
# http://help.agi.com/stkdevkit/index.htm#stkObjects/ObjModPythonCodeSamples.htm

# Enter and run the lines of code to compute access between the satellite and the target

access = satellite.GetAccessToObject(target)
access.ComputeAccess();

In [10]:
# Retrieve and view the access data in Python.

accessDP         = access.DataProviders.Item('Access Data')

results          = accessDP.Exec(scenario.StartTime, scenario.StopTime)

accessStartTimes = results.DataSets.GetDataSetByName('Start Time').GetValues()

accessStopTimes  = results.DataSets.GetDataSetByName('Stop Time').GetValues()

print(accessStartTimes,accessStopTimes)

['19 Jan 2021 08:52:05.288036424', '19 Jan 2021 10:32:43.928486778', '19 Jan 2021 12:14:27.919268383', '19 Jan 2021 20:22:35.534646016', '19 Jan 2021 21:59:55.028006652', '19 Jan 2021 23:41:34.439013271', '20 Jan 2021 01:28:52.071777377'] ['19 Jan 2021 09:04:59.846966898', '19 Jan 2021 10:48:24.111646478', '19 Jan 2021 12:27:32.530493254', '19 Jan 2021 20:32:53.183823836', '19 Jan 2021 22:15:11.647053212', '19 Jan 2021 23:56:18.176055050', '20 Jan 2021 01:35:17.605111331']


In [11]:
# Note: Generating the Start & Stop times in Python can 
# also be done using the following lines of code.
accessIntervals = access.ComputedAccessIntervalTimes

dataProviderElements = ['Start Time', 'Stop Time']

for i in range(0,accessIntervals.Count):
    times = accessIntervals.GetInterval(i)
    print(times)

('19 Jan 2021 08:52:05.288', '19 Jan 2021 09:04:59.847')
('19 Jan 2021 10:32:43.928', '19 Jan 2021 10:48:24.112')
('19 Jan 2021 12:14:27.919', '19 Jan 2021 12:27:32.530')
('19 Jan 2021 20:22:35.535', '19 Jan 2021 20:32:53.184')
('19 Jan 2021 21:59:55.028', '19 Jan 2021 22:15:11.647')
('19 Jan 2021 23:41:34.439', '19 Jan 2021 23:56:18.176')
('20 Jan 2021 01:28:52.072', '20 Jan 2021 01:35:17.605')


In [12]:
# Retrieve and view the altitude of the satellite during an access interval.

satelliteDP       = satellite.DataProviders.Item('LLA State')

satelliteDP2      = satelliteDP.Group.Item('Fixed')

rptElements       = ['Time', 'Lat', 'Lon', 'Alt']

satelliteDPTimeVar = satelliteDP2.ExecElements(accessStartTimes,accessStopTimes, 60, rptElements)

satelliteAltitude = satelliteDPTimeVar.DataSets.GetDataSetByName('Alt').GetValues()

print(satelliteAltitude)

[841.8893302305768, 841.1776873978358, 840.3308531003345, 839.3648047514721, 838.2942511532125, 837.1355119267819, 835.9062743325505, 834.6253283042813, 833.3122833577369, 831.9872713810131, 830.6706396209271, 829.382638437036, 828.1431085920332, 827.0687084781512]


In [13]:
# You have just completed the STK integration with Python tutorial using Jupyter Notebooks. 
# Don't forget to save your work. 

# Note: To save your scenario you will want to create a new directory 
# and then tell python to save all the scenario files there.
import os
os.mkdir('Python_STK_Training')

# This will create a new folder in your scenario directory. 
# Open a file explorer window and copy the file path. The next 
# line will use your custom path.

directory = os.getcwd() + "\Python_STK_Training"
root.ExecuteCommand('Save / * \"' + directory + '\"');

# With your scenario safely saved, you can close out of STK. 
# Workflows like these can be expanded and automated for quickly building and analyzing missions.